In [ ]:
# Import required libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from moviepy.editor import VideoFileClip
from IPython.display import HTML

debug = True




In [ ]:
#Set list of configuration parameters
mag_sobel_threshold_min = 10
mag_sobel_threshold_max = 180

dir_sobel_threshold_min = 0.5
dir_sobel_threshold_max = 0.8

hls_threshold_low = 50
hls_threshold_high = 255

#First frame of video
first_frame_in_video = 1

#Window for looking for pixels
window_margin = 80
num_windows = 9
min_pixels_windows = 45
num_pixels_for_polyfit_search = 150

#Calibration 
warp_src = np.float32([[297,649],
                  [991,649],
                  [619,434],
                  [656,434]])
        


warp_dst = np.float32([[458,720],
                  [823.5,720],
                  [458,0],
                  [823.5,0]])


 
left_fit = None
right_fit = None
lower_limit = 100



In [ ]:
# Help for Debug
def print_debug(x):
    if (debug == True):
        print (x)
        
def print_debug_2(x,y):
    if (debug == True):
        print (x,y)
        
        
def print_debug_3(x,y,z):
    if (debug == True):
        print (x,y,z)

        
#Show img
def plot_image(img1,img2, img2_title, img2_cmap='BrBG'):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(img2, cmap=img2_cmap)
    ax2.set_title(img2_title, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

def plot_3_images(img1,img2, img3,  img1_title ='1', img2_title = '2', img3_title= '3', 
                  img1_cmap= 'BrBG', img2_cmap='BrBG', img3_cmap='BrBG'):
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Original Image', fontsize=50)
    
    ax2.imshow(img2, cmap=img2_cmap)
    ax2.set_title(img2_title, fontsize=50)
    
    ax3.imshow(img3, cmap=img3_cmap)
    ax3.set_title(img3_title, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    



In [ ]:
#Function to calibrate a camera
# gray = A gray checssboard image
# nx,ny -> No of x and y interections
# start_x, start_y -> x, y of initial point
# square_length -> Width of a square
def calibrateCamera(gray, nx,ny,start_x, start_y, square_length):
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if (ret == True):
        
        #objpoints = Vector of vector of POint3f (# of Views, # of POints, 3d points..
        #img points = Vector of vector of points (# of Views, #of Points, 2d points)
        # We use 1 View
        #
        # We use same # of points - num_calibration_x_points * num_calibration_y_points
        num_calibration_points = 3*3
        objpoints = np.zeros((num_calibration_points,3), dtype = 'float32')
        imgpoints = np.zeros((num_calibration_points,2), dtype = 'float32')
        
        index = 0
        for row in (0, int((ny)/2), ny-1):
            for col in (0, int((nx)/2), nx-1):
                corners_index = (row*nx)+ col
                imgpoints[index][0] = corners[int(corners_index),0,0]
                imgpoints[index][1] = corners[int(corners_index),0,1]
                
                #Fill up objpoints
                objpoints[index] = [start_x + square_length *col, start_y+row*square_length,0.]
                index = index + 1
                
        objlist = [objpoints]
        imglist = [imgpoints]
        #Get distortion matrix
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objlist, imglist, gray.shape[::-1], None, None)
        return ret,mtx,dist,rvecs,tvecs

    else:
        print ("ret is false, findChessBoardCorners failed")
        return False,None,None,None,None
    


In [ ]:
#List of functions for Thresholding (filtering ) an image
def get_sobeled_binary_absolute(gray, thresh_min, thresh_max):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobel = np.absolute(np.sqrt((sobelx**2) + (sobely**2)))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    abs_sobelx = np.absolute(sobelx)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    abs_sobely = np.absolute(sobely)
    scaled_sobely = np.uint8(255*abs_sobelx/np.max(abs_sobely))
    
    sxbinary = np.zeros_like(scaled_sobel)
    #sxbinary[(scaled_sobelx >= thresh_min) & (scaled_sobelx <= thresh_max)] = 255
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max) & (scaled_sobelx > 5) ] = 255
    return sxbinary

#Not used
def get_sobeled_binary_gradient(gray, thresh_min, thresh_max):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=15)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=15)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    gradient = np.arctan2(abs_sobely, abs_sobelx)
    binary_output = np.zeros_like(gradient).astype('uint8')
    binary_output[(gradient >= thresh_min) & (gradient < thresh_max)] = 255
    return binary_output

#Not used
def hls_select_with_sobel(img, thresh_min, thresh_max):
    # 1) Convert to HLS color space
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    sobelx = cv2.Sobel(S, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    binary_output = np.zeros_like(S)
    binary_output[ ( scaled_sobelx >thresh_min) & (  scaled_sobelx <= thresh_max)] = 255
    #binary_output = np.copy(img) # placeholder line
    return binary_output

#Not used
def hls_select(img, thresh_min, thresh_max):
    # 1) Convert to HLS color space
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[   (S >thresh_min) & ( S <= thresh_max)] = 255
    #binary_output = np.copy(img) # placeholder line
    return binary_output

#Not used
def hsv_select(img, thresh_min, thresh_max):
    # 1) Convert to HLS color space
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    H = hls[:,:,0]
    S = hls[:,:,1]
    V = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_yellow = np.zeros_like(S)
    binary_white = np.zeros_like(S)
    binary_yellow[(H>15) & ( H< 45) & (S >10) & ( S <= 255) & (V >150) & (V <= 255)] = 1
    binary_white[(H>0) & ( H< 180) & (S >0) & ( S <= 20) & (V >50) & (V <= 255)] = 1
    
    binary_output[(binary_yellow ==1) | (binary_white == 1)] = 1
   
    #binary_output = np.copy(img) # placeholder line
    return binary_output

def rgb_select(img, gray, thresh_min, thresh_max):
    # 1) Convert to HLS color space
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    Y = np.zeros_like(R)
    W = np.zeros_like(R)
    Y_or_W = np.zeros_like(R)
    Y[(R/B> 1.2) & (G/B>  1.2)] = 255
    W[(R/B <  1.2) & (G/B <   1.2) & (R/B >0.8) & (G/B > 0.8) & (gray >195)] = 255
    Y_or_W [(Y==255) | (W == 255)] = 255
    #binary_output [ B < 80] = 1
    #binary_output = np.copy(img) # placeholder line
    return Y_or_W


def get_combined_color_gradient_filter (img,gray):
    
    #Try with absolute binary
    mag_binary = get_sobeled_binary_absolute(gray, mag_sobel_threshold_min,mag_sobel_threshold_max)
    #Try with gradient
    dir_binary = get_sobeled_binary_gradient( gray,0.5, 0.7)
    
    
    #Try with color
    hls_binary = hls_select(img, hls_threshold_low, hls_threshold_high)
    rgb_binary = rgb_select(img, gray, 230,255)
    
    hls_sobel_binary = hls_select_with_sobel(img,10,255)
    hsv_binary = hsv_select(img, 0,76)

                    
    #Combined
    combined = np.zeros_like(dir_binary)
   # combined [(mag_binary==255)   & (rgb_binary == 255) |(hls_binary == 255)] = 255
    combined [(mag_binary==255)   & (rgb_binary == 255)] = 255
    
    return combined, mag_binary, dir_binary, hls_binary, hsv_binary, rgb_binary

#Get view from top
def getViewFromTop(orig_img,img_to_warp,src,dst):

    img_size = (img_to_warp.shape[1], img_to_warp.shape[0])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img_to_warp, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [ ]:
def hist(img):
    # TO-DO: Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[np.uint32(img.shape[0]*3/4):, :]
    
    
    
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half,axis=0)
    
    #print(histogram.shape[0])
    return histogram

#Find average x value of a set of non zero values in a window
# if number of non zero values > minpix, move leftx_base to new average
# Else return original base
# Also return a list of x *2 which has all the points inside window
def  find_average_x_and_points_in_window( base, window_lower_left, window_upper_right, 
                                         view_from_top_non_zeros,minpix):
    num_points = 0
    total_x = 0
    lane_inds = []
    empty_lane_inds = []
    for index in range(len(view_from_top_non_zeros[0])):
        y = view_from_top_non_zeros[0][index]
        x = view_from_top_non_zeros[1][index]
        if ((y > window_lower_left[1]) & ( y < window_upper_right[1])):
            if ((x > window_lower_left[0]) & ( x < window_upper_right[0])):
                num_points = num_points + 1
                total_x = total_x + x
                lane_inds.append([x,y])
    
    if (num_points > minpix):
        new_base = np.int(total_x/num_points)
        delta = new_base - base
        base = base + 1 *delta
     
    #Don't worry about stray points
    #if (num_points < 5):
        #return base, empty_lane_inds
    return base, lane_inds

#Fit a polynomial for a set of x,y
# Input - nonzero elements is a 2 * X array, 1st array having x and 2nd array having y
def fit_polynomial(nonzero_elements, order = 2):
    if (len (nonzero_elements) > 1):
        return np.polyfit(nonzero_elements[1], nonzero_elements[0],order)
    else :
        return np.polyfit(np.array([0,5]), np.array([0,5]), order)

#Find the lane pixels of a lane
# img -> Original image (w *h *3)
# gray -> gray image (w *h)
# view_from_top -> View from top of gray image (w * h)
# 1. Find histogram of view_from_top
# 2. Get base of 2 lines from histogram (leftx_base, rightx_base)
# 3. Create n sliding windows
# 4. Recenter windows as per actual lane markings
# 5 . Find all points within the rectangles
# 6.  Fit a polygon for both left and right lanes
# 7. Return the polygon, left_lane_inds, right_lane_inds
def find_lane_pixels_and_fit_poly (img, gray, view_from_top):
    histogram = hist(view_from_top)
    
    #Get left and right base
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[midpoint -250:midpoint]) + (midpoint-250)
    rightx_base = np.argmax(histogram[midpoint +100 :midpoint+250]) + midpoint + 100
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin =  window_margin
    # Set minimum number of pixels found to recenter window
    minpix = 45
    
    good_left_lane_inds = []
    good_right_lane_inds = []
    delta_left = 0
    delta_right = 0
    
    
    #Find nonzeros in each window and adjust window
    #Non zeros outputs 2 arrays - first for y, second for x
    # these represent points Eg: view_from_top_non_zeros[0][5]=100
    # view_from_top_non_zeros[1][5]= 200 
    #  (x,y) = (100,200) is a non zero value
    view_from_top_non_zeros = np.nonzero(view_from_top)


    #Find the nonzeros in each window and move window
    #Also, find the good_left_lane_inds and good_right_lane_inds
    # These are list of x *2 (Eg: List of 1000 * 2 )
    
    window_height = img.shape[0]/nwindows
    for window in reversed(range(nwindows)):
        window_lower_left = (np.uint32(leftx_base-margin), np.uint32(window *window_height))
        window_upper_right = (np.uint32(leftx_base+margin), np.uint32((window+1) * window_height))
        
        leftx_base_prev = leftx_base
        leftx_base, lane_inds = find_average_x_and_points_in_window(leftx_base,  window_lower_left, 
                                                                    window_upper_right,
                                                                    view_from_top_non_zeros,minpix)
        #Reduce margin as we go along
        margin = margin - 5
        
        
        good_left_lane_inds.extend(lane_inds)
        
        
        ploty = 0
        if (len(good_left_lane_inds) > 150):
            left_fit_loc  = fit_polynomial(np.transpose(good_left_lane_inds),3)
            ploty = window_lower_left[1] - 0#window_height/2
            leftx_base = left_fit_loc[0]*ploty**3 + left_fit_loc[1]*ploty**2 + left_fit_loc[2]*ploty + left_fit_loc[3]
            
    
        leftx_base = max(margin, leftx_base)
        leftx_base = min (leftx_base, view_from_top.shape[1]- margin)
        
        delta_left = leftx_base  -leftx_base_prev
        
       
            
        if (delta_left > margin):
            leftx_base = leftx_base_prev + margin
        if (delta_left < -margin):
            leftx_base = leftx_base_prev - margin
            
        delta_left = leftx_base  -leftx_base_prev
        
        
        cv2.rectangle(view_from_top, window_lower_left, window_upper_right, 255,8)

        # Do same as above for right lane too
        window_lower_left = (np.uint32(rightx_base-margin), np.uint32(window *img.shape[0]/nwindows))
        window_upper_right = (np.uint32(rightx_base+margin), np.uint32((window+1) * img.shape[0]/nwindows))
        rightx_base_prev = rightx_base
        #print("delta_right, rightx_base ,window y is  ", delta_right,rightx_base, window_lower_left[1],
         #    window_upper_right[1])
        rightx_base, lane_inds = find_average_x_and_points_in_window (rightx_base,  
                                                                      window_lower_left, window_upper_right, 
                                                                      view_from_top_non_zeros,minpix)
        
        good_right_lane_inds.extend(lane_inds)
        
        if (len( good_right_lane_inds) > 150):
            right_fit_loc  = fit_polynomial(np.transpose(good_right_lane_inds),3)
            ploty = window_lower_left[1] - 0#window_height/2
            rightx_base = right_fit_loc[0]*ploty**3 + right_fit_loc[1]*ploty**2 + right_fit_loc[2]*ploty + right_fit_loc[3]
            
         
       
        rightx_base = max(margin, rightx_base)
        rightx_base = min (rightx_base, view_from_top.shape[1]- margin)
        
        delta_right = rightx_base  - rightx_base_prev
        if (delta_right > margin):
            rightx_base = rightx_base_prev + margin
        if (delta_right < -margin):
            rightx_base = rightx_base_prev - margin
        delta_right = rightx_base  - rightx_base_prev
        
       
        
        #print ("delta, nwindows are ", rightx_base - orig_rightx_base, window)
        cv2.rectangle(view_from_top, window_lower_left, window_upper_right, 255,20)
    

    for index in range(len(good_left_lane_inds)):
        cv2.circle(view_from_top, (good_left_lane_inds[index][0], good_left_lane_inds[index][1]),
                   np.int(10), (255,255,255),np.int(10))
        
    for index in range(len(good_right_lane_inds)):
        cv2.circle(view_from_top, (good_right_lane_inds[index][0], good_right_lane_inds[index][1]),
                   np.int(10), (255,255,255),np.int(10))
    
    #Fit polynomials     
    left_fit  = fit_polynomial(np.transpose(good_left_lane_inds),2)
    right_fit  = fit_polynomial(np.transpose(good_right_lane_inds),2)
    

    
    #Return
    return good_left_lane_inds, good_right_lane_inds, left_fit, right_fit, histogram

def find_lane_pixels_and_fit_poly_repeat (img, gray, view_from_top,left_fit_param,right_fit_param):
    
    
    histogram = hist(view_from_top)
    
    #Get left and right base
    midpoint = np.int(histogram.shape[0] *1/2)
    leftx_base = np.argmax(histogram[midpoint -200:midpoint]) + (midpoint-200)
    rightx_base = np.argmax(histogram[midpoint:midpoint+200]) + midpoint
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    #nwindows = 9
    # Set the width of the windows +/- margin
    margin =  window_margin/4
    # Set minimum number of pixels found to recenter window
    #minpix = 50
    
    good_left_lane_inds = []
    good_right_lane_inds = []
    
    
    #Find nonzeros in each window and adjust window
    #Non zeros outputs 2 arrays - first for y, second for x
    # these represent points Eg: view_from_top_non_zeros[0][5]=100
    # view_from_top_non_zeros[1][5]= 200 
    #  (x,y) = (100,200) is a non zero value
    view_from_top_non_zeros = np.nonzero(view_from_top)
    nonzeroy =  view_from_top_non_zeros[0]
    nonzerox =  view_from_top_non_zeros[1]
    



    for i in range(len(nonzeroy)):
        y = nonzeroy[i]
        x = left_fit_param[0] * (y **2) + left_fit_param[1] * y + left_fit_param[2]
        xright = right_fit_param[0] * (y **2) + right_fit_param[1] * y + right_fit_param[2]
        if (abs(x-nonzerox[i]) < margin):
            good_left_lane_inds.append([nonzerox[i],y])
        elif  (abs(xright-nonzerox[i]) < margin):
            good_right_lane_inds.append([nonzerox[i], y])
            
    for index in range(len(good_left_lane_inds)):
        cv2.circle(view_from_top, (good_left_lane_inds[index][0], good_left_lane_inds[index][1]),
                   np.int(10), (255,255,255),np.int(10))
        
    for index in range(len(good_right_lane_inds)):
        cv2.circle(view_from_top, (good_right_lane_inds[index][0], good_right_lane_inds[index][1]),
                   np.int(10), (255,255,255),np.int(10))
    
             
    #Fit polynomials    
    left_fit_local  = fit_polynomial(np.transpose(good_left_lane_inds),2)
    right_fit_local  = fit_polynomial(np.transpose(good_right_lane_inds),2)
    
    
    
    #Return
    return good_left_lane_inds, good_right_lane_inds, left_fit_local, right_fit_local, histogram


    
#find curvature
# a , b are coefficients of a polynomial x=  a*y^2 + b *y + C
# This is calculated for a given value of y_input
def calculate_curvature ( y_input,a ,b):
    num = ( 1 + (2*a*y_input + b)**2) ** (3/2)
    den = abs ( 2 *a)
    return (num//den)


In [ ]:
#Function to detect lines in an image
# outputs an image which has the original imposed with lanes on top of them
def proces_pixel(img, fname = 'default',write_output = 0):
  

    #Define global vaiables

    global left_fit
    global right_fit
    global first_frame_in_video
  

    # Undisort image
    # Get gray scale
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)

    #Get thresholded image and use only a certain ROI
    filtered ,mag_binary, dir_binary, hls_binary, hsv_binary, rgb_binary = get_combined_color_gradient_filter (undist,gray)
    
    #use ROI
    # ROI is all elements from 100 pixels from corner to mid of the image
    height = gray.shape[0] - 1
    width =  gray.shape[1] - 1
    midpoint_y = np.int(height/2)
    midpoint_x = np.int (width/2)
    
    
    left_x_roi = 100
    right_x_roi = gray.shape[1]-100
    
    
    #Top  - Filter out anything on top half
    filtered[0: midpoint_y,:] = 0
    
    #Left - Calculate slope and y_intercept
    # Line connecting 2 points (midpoint_y, midpoint_x)  and ( height, left_x_roi)
    slop = ( midpoint_y - height)/ ( midpoint_x - left_x_roi ) 
    # c = y1 - m * x1
    c = height - slop * left_x_roi    
    for xleft in  range(left_x_roi, midpoint_x  ,1):
        y = int(slop * xleft + c)
        filtered[0:y, xleft] = 0
        
    #Filter out all the pixels in left from 0 to left_x_roi    
    filtered[:,0:left_x_roi] =0  
    
    # Now ROI for right side
    slop = ( midpoint_y - height)/ ( midpoint_x - right_x_roi ) 
    # c = y1 - m * x1
    c = height - slop * right_x_roi    
    for xright in  range(midpoint_x  ,right_x_roi, 1):
        y = int(slop * xright + c)
        filtered[0:y, xright] = 0
        
    #Filter out all the pixels in right from 0 to left_roi    
    filtered[:,right_x_roi: width] =0
   
    
    
    
    
    # Get warped gray, undisortedand filtered image
    view_from_top_gray ,M, Minv = getViewFromTop(img,gray, warp_src, warp_dst)
    view_from_top_undist ,M, Minv = getViewFromTop(img,undist,warp_src, warp_dst)
    view_from_top_filtered, M , Minv = getViewFromTop(img,filtered,warp_src,warp_dst)    
    
    view_from_top_filtered[0:300,:] = 0
    
    view_from_top_filtered_2 = np.zeros_like(view_from_top_filtered)
    view_from_top_filtered_2 [(view_from_top_filtered != 0 )] = 255
     
   
    # - Find the left , lane and the left and right polygons
    #First , make a copy so that you don't overwrite the originalview_from_top
    view_from_top_rectangle = view_from_top_filtered.copy()

    # For 2nd frame of video onwards, get line and points of land indicators based on previous frames
    ff = first_frame_in_video
    if ( first_frame_in_video == 0):
        good_left_lane_inds, good_right_lane_inds, left_fit_out, right_fit_out, histog  =  find_lane_pixels_and_fit_poly_repeat (img,
                                                                                                              gray, 
                                                                                                              view_from_top_rectangle,
                                                                                                                 left_fit,
                                                                                                                 right_fit)

        left_fit = left_fit_out
        right_fit = right_fit_out
        print_debug("Using previous frame info")
        if ((len (good_left_lane_inds) < lower_limit ) | (len(good_right_lane_inds) < lower_limit)):
            first_frame_in_video = 1
            print (" no good lanes, restarting")

    # For first frame of video ( or any image) , get line and points of lane indicators
    if(first_frame_in_video == 1):
        first_frame_in_video = 0
        ff = 1
        good_left_lane_inds, good_right_lane_inds, left_fit, right_fit, histog  =  find_lane_pixels_and_fit_poly (img,
                                                                                                              gray, 
                                                                                                              view_from_top_rectangle)
        
        if (len (good_left_lane_inds) < lower_limit ):
            print ("No good left lane")
            good_left_lane_inds = []
            
        if (len (good_right_lane_inds) < lower_limit ):
            print ("No good right lane")
            good_right_lane_inds = []
            
            
      
        
    #For debugging    
    view_from_top_filtered_with_rectangle = cv2.addWeighted(view_from_top_filtered , 0.1, view_from_top_rectangle, 0.9, 0)
    
    #Create x,y points from polynomial fit
    ploty = np.linspace(0,720,100)
    
    #Only draw lanes from points which we found 
    good_left_lane_inds_asarray = np.asarray(good_left_lane_inds)
    y_list = good_left_lane_inds_asarray[:,1]
    if (len(good_left_lane_inds) > 0):
        y_min = np.min(y_list)
    else:
        y_min = 0
        print ("No good left lane!")
        
  
    ploty = np.linspace(y_min,720,100)
    left_fitx= left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx= right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    

    # Draw 2 lines an  fill them
    # Some tricks here
    # Left points are obtained by dstack which converts 2 (100x1) arrays into 1 (1x100x2) arrays
    # So [5,10]  and [500,100] becomes [[[5,500], [10,100]]]

    #For pts right we do some tricks so that the lines are from bottom to top  y =(720 to 0)
    #This is done for drawing a polygon - where bottom of left lane is connected to bottom of right lane
    # The trick here is - first create a 2D array - which convers 2 100x1 into 1 (100x2)
    # Then flip it upside down - so it is still 100x2, except first row is what was last row
    pts_left = np.dstack ((np.int_(left_fitx), np.int_ (ploty)))
    pts_right = np.transpose(np.vstack ((np.int_(right_fitx), np.int_ (ploty))))
    pts_right_flipud = np.flipud(pts_right)

    pts = np.hstack((pts_left, [pts_right_flipud]))

 
    #Draw left lanes and right lanes and fill polygon
    view_from_top_zeros = np.zeros_like(view_from_top_gray)
    view_from_top_left_lane = np.zeros_like(view_from_top_gray).astype('uint8')
    view_from_top_right_lane =  np.zeros_like(view_from_top_gray).astype('uint8')
    view_from_top_filled = np.zeros_like(view_from_top_zeros).astype('uint8')
    cv2.polylines(view_from_top_left_lane, pts_left, False, (254,0,0),10)
    cv2.polylines(view_from_top_right_lane, [pts_right_flipud], False, (254,254,0),10)
    cv2.fillPoly (view_from_top_filled, pts, 255)

    #Now put the lanes back into actual image
    img_size = (img.shape[1], img.shape[0])
   
    view_from_top_left_lane_inverted = cv2.warpPerspective(view_from_top_left_lane, Minv,img_size, flags=cv2.INTER_LINEAR)
    view_from_top_right_lane_inverted = cv2.warpPerspective(view_from_top_right_lane, Minv,img_size, flags=cv2.INTER_LINEAR)
    view_from_top_filled_inverted = cv2.warpPerspective(view_from_top_filled, Minv, img_size, flags=cv2.INTER_LINEAR)
    
  
    #Create an image of left, right and filled lanes in normal view
    view_from_top_color = np.dstack(( view_from_top_left_lane_inverted.astype('uint8'),view_from_top_filled_inverted.astype('uint8'),
                                      view_from_top_right_lane_inverted.astype('uint8')))
    view_from_top_lanes_color = np.dstack(( view_from_top_left_lane,view_from_top_filled,
                                      view_from_top_right_lane))
    
    #For debugging and output files of intermediate steps
    if (debug):
        view_from_top_noninverted_color = np.dstack(( view_from_top_left_lane, view_from_top_filtered_with_rectangle,
                                     view_from_top_right_lane))
        view_from_top_filtered_color = np.dstack((view_from_top_filtered_2 ,view_from_top_gray, view_from_top_zeros))
        view_from_top_filtered_color2 = np.dstack((dir_binary, hsv_binary, rgb_binary))
        view_from_top_filtered_color2[0:430,:] = (0,0,0)
  

    #We are all set! Creae the final output
    imposed_img = cv2.addWeighted(img,0.5,view_from_top_color,0.5,0)

    #Calculate curvature and put text
    ym_per_pix = 30/720
    xm_per_pix = 3.7/700
    left_fit_in_meters  = fit_polynomial (np.vstack((left_fitx *xm_per_pix, ploty*ym_per_pix)),2)
    right_fit_in_meters  = fit_polynomial(np.vstack((right_fitx *xm_per_pix, ploty*ym_per_pix)),2)
    left_curvature = calculate_curvature ( np.max(ploty*ym_per_pix),left_fit_in_meters[0]  ,left_fit[1])
    right_curvature = calculate_curvature ( np.max(ploty*ym_per_pix),right_fit_in_meters[0]  ,right_fit[1])
    cv2.putText(imposed_img, "curv(left)="+ str(left_curvature) , (100,100),cv2. FONT_HERSHEY_DUPLEX,3, (255,255,255),5)
    cv2.putText(imposed_img, "curv(right)="+ str(right_curvature),(100,200),cv2. FONT_HERSHEY_DUPLEX,3, (255,255,255),5)
    
    if (debug):
        cv2.putText(imposed_img, "FF =" + str(ff), (100,300),cv2. FONT_HERSHEY_DUPLEX,3, (255,255,255),5)
    
    
    #Calculate centers of lane and camera and draw
    max_ploty = np.max(ploty)-1
    left_fit_bottom = left_fit[0]*max_ploty**2 + left_fit[1]*max_ploty + left_fit[2]
    right_fit_bottom = right_fit[0]*max_ploty**2 + right_fit[1]*max_ploty + right_fit[2]
    lane_center = (np.int((left_fit_bottom + right_fit_bottom)/2),np.int(np.max(ploty))-10)
    camera_center = (np.int(img.shape[1]/2), np.int(img.shape[0])-10)
    cv2.circle(imposed_img, lane_center, np.int(5), (255,255,255),np.int(10))
    cv2.circle(imposed_img, camera_center, np.int(5), (255,0,0),np.int(10))
    
    if (debug):
        plot_3_images(filtered, imposed_img, view_from_top_noninverted_color,"For debug")
    if (debug & (write_output==1)):
        fname_base =  os.path.basename(fname)
        fname_filtered = 'output_images/thresholded_images/' + fname_base + 'filtered.jpg'
        fname_filtered_warped = 'output_images/warped_images/' + fname_base + 'filtered_warped.jpg'
        fname_lanes = 'output_images/lanes_from_top/' + fname_base + 'lanes_top_view.jpg'
        fname_output =  'output_images/final_images/' + fname_base + 'final_image.jpg'
        cv2.imwrite (fname_filtered, filtered)
        cv2.imwrite (fname_filtered_warped, view_from_top_filtered)
        cv2.imwrite (fname_lanes, cv2.cvtColor(view_from_top_lanes_color,cv2.COLOR_BGR2RGB))
        new_imposed = cv2.cvtColor(imposed_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite (fname_output, new_imposed)
    return imposed_img

In [ ]:
import glob,os
#Main function 

for fname in glob.glob("camera_cal/calibration1.jpg"):
    nx=9
    ny=5
    calibration_img = cv2.imread(fname)
    #print (calibration_img)
    # Convert to grayscale
    gray = cv2.cvtColor(calibration_img, cv2.COLOR_BGR2GRAY)
    print (os.path.basename(fname))

    ret, mtx, dist, rvecs, tvecs = calibrateCamera(gray, nx,ny,137,96,121)
    #undistort
    
for fname in glob.glob('camera_cal/*.jpg'):
    calibration_img = cv2.imread(fname)
    #print (calibration_img)
    # Convert to grayscale
    gray = cv2.cvtColor(calibration_img, cv2.COLOR_BGR2GRAY)
    undist = cv2.undistort(calibration_img, mtx, dist, None, mtx)
    output_fname = 'undistorted_chess' + os.path.basename(fname)
    cv2.imwrite( output_fname,undist)
    plot_image(calibration_img,undist,fname)






In [ ]:
#Do lane detection for images
debug = True

#hls_threshold_low = 200
#hls_threshold_high = 255
#write_output = 1

for fname in glob.glob("test_images/*.jpg"):
    first_frame_in_video=1
    orig_img = cv2.imread(fname)
    img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    imposed_img = proces_pixel(img, fname, write_output=0)
    plot_image( img, imposed_img,fname)
    
    




In [ ]:
#Lane detection for Videos
hls_threshold_low = 0
hls_threshold_high = 0
debug=False
video_output = 'outputVideo6.mp4'
first_frame_in_video = 1

#clip1 = VideoFileClip("project_video.mp4").subclip(22,24)
clip1 = VideoFileClip("project_video.mp4")
output_clip = clip1.fl_image(proces_pixel)
output_clip.write_videofile(video_output, audio=False)
output_clip.reader.close()
clip1.reader.close()

output_clip.reader.close()
    